In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_excel("/content/drive/MyDrive/00. Drive PC/1.STIS/SEMNAS 2024/ihsg detik aisy.xlsx")
df

,Judul,Tanggal,Link,Berita,Kode,Label
0,"ISHG Dibuka di Zona Hijau, Kembali ke Zona 7.2...",22 Mei 2024 09:13,https://finance.detik.com/bursa-dan-valas/d-73...,Indeks Harga Saham Gabungan (IHSG) pagi ini di...,1,ihsg naik
1,IHSG Dibuka di Zona Hijau Usai Kemarin Melempem,21 Mei 2024 09:12,https://finance.detik.com/bursa-dan-valas/d-73...,Indeks Harga Saham Gabungan (IHSG) pagi ini di...,1,ihsg naik
2,"Dibuka di Zona Hijau, IHSG Sore Ini Ditutup Me...",20 Mei 2024 16:31,https://finance.detik.com/bursa-dan-valas/d-73...,Indeks Harga Saham Gabungan (IHSG) menutup per...,0,ihsg turun
3,"IHSG Begitu Cerah, Awali Pekan di Zona Hijau",20 Mei 2024 09:10,https://finance.detik.com/bursa-dan-valas/d-73...,Indeks Harga Saham Gabungan (IHSG) mengawali p...,1,ihsg naik
4,Akhir Pekan IHSG Betah di Zona Hijau,17 Mei 2024 17:08,https://finance.detik.com/bursa-dan-valas/d-73...,Indeks harga saham gabungan (IHSG) sepanjang h...,1,ihsg naik
...,...,...,...,...,...,...
2459,"Rupiah Keok, IHSG Melempem ke 6.119",05 Des 2018 12:29,https://finance.detik.com/bursa-dan-valas/d-43...,Indeks Harga Saham Gabungan (IHSG) ditutup di ...,0,ihsg turun
2460,"Rupiah Keok, IHSG Menghijau Seharian",04 Des 2018 16:27,https://finance.detik.com/bursa-dan-valas/d-43...,Indeks Harga Saham Gabungan (IHSG) ditutup di ...,1,ihsg naik
2461,"Lanjutkan Penguatan, IHSG Parkir di 6.134",04 Des 2018 12:50,https://finance.detik.com/bursa-dan-valas/d-43...,Sementara nilai tukar dolar Amerika Serikat (A...,1,ihsg naik
2462,"Sepanjang Hari Lesu, IHSG Berakhir di 6.056",30 Nov 2018 16:50,https://finance.detik.com/bursa-dan-valas/d-43...,Indeks Harga Saham Gabungan (IHSG) sepanjang h...,0,ihsg turun


In [4]:
df['Label'].value_counts()

Label
ihsg naik     1394
ihsg turun    1070
Name: count, dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2464 entries, 0 to 2463
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Judul    2464 non-null   object
 1   Tanggal  2464 non-null   object
 2   Link     2464 non-null   object
 3   Berita   2464 non-null   object
 4   Kode     2463 non-null   object
 5   Label    2464 non-null   object
dtypes: object(6)
memory usage: 115.6+ KB


# Preprocessing

In [6]:
!pip install emoji
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 7.7 MB/s eta 0:00:00


In [7]:
import pandas as pd
import numpy as np
import nltk
import string
import re
import sklearn
import emoji

In [8]:
# Menemukan baris dengan tipe data float
float_rows = df[df['Berita'].apply(lambda x: isinstance(x, float))]

print("Baris dengan tipe data float:")
print(float_rows)

Baris dengan tipe data float:
Empty DataFrame
Columns: [Judul, Tanggal, Link, Berita, Kode, Label]
Index: []


In [ ]:
# Menemukan baris dengan string "nan"
nan_string_rows = df[df['Berita'] == 'nan']

print("Baris dengan string 'nan':")
print(nan_string_rows)

Baris dengan string 'nan':
Empty DataFrame
Columns: [Judul, Tanggal, Link, Berita, Kode, Label_1, Label_2, Label Fix, Unnamed: 8]
Index: []


In [9]:
df.isna().sum()

Judul      0
Tanggal    0
Link       0
Berita     0
Kode       1
Label      0
dtype: int64

In [10]:
df.dropna(axis=1)
df.dropna(how='all')
df = df.dropna(subset=['Kode'])

In [11]:
df.isna().sum()

Judul      0
Tanggal    0
Link       0
Berita     0
Kode       0
Label      0
dtype: int64

# INI UNTUK BERT

In [ ]:
def remove_specific_words_and_lowercase(text, stopwords):
    if isinstance(text, str):
        words = text.split()  # Split text into a list of words
        filtered_words = [word for word in words if word.lower() not in stopwords]  # Remove stopwords
        return ' '.join(filtered_words).lower()  # Join remaining words and convert to lowercase
    else:
        return text  # Return the text as is if it's not a string
# Daftar stopwords yang ingin dihapus
stopwords = ['scroll', 'to', 'continue', 'with', 'content']

df['Cleaned Berita'] = df['Berita'].apply(remove_specific_words_and_lowercase, stopwords=stopwords)

In [ ]:
# Menemukan baris dengan string "nan"
nan_string_rows = df[df['Berita'] == 'nan']

print("Baris dengan string 'nan':")
print(nan_string_rows)

Baris dengan string 'nan':
Empty DataFrame
Columns: [Judul, Tanggal, Link, Berita, Kode, Label_1, Label_2, Label Fix, Unnamed: 8, Cleaned Berita]
Index: []


In [ ]:
df.to_excel("data_bert_all.xlsx")

# INI UNTUK MACHINE LEARNING BIASA

In [12]:
def preprocess(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text, flags=re.MULTILINE)
    # Remove usernames (assuming usernames start with @)
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove punctuation, symbols, numbers, and whitespace
    text = re.sub(r'[^\w\s\d+\s+]', '', text).strip()
    # hapus emoji
    text = emoji.demojize(text, delimiters=("", ""))

    text = text.lower() #lowercase text
    text=text.strip() #Menghapus leading/trailing whitespace
    text = re.sub('@[^\s]+','atUser',text) #mengubah @user menjadi atUser
    text = re.sub(r'#([^\s]+)', r'\1', text) #menghapus hashtag di depan suatu kata
    text= re.compile('<.*?>').sub('', text) #Menghapus HTML tags/markups
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    #Replace punctuation with space. Careful since punctuation can sometime be useful
    text = re.sub('\s+', ' ', text) #Menghapus extra space dan tabs
    text = re.sub(r'\[[0-9]*\]',' ',text) #[0-9] matches any digit (0 to10000...)
    text= re.sub(r'[^\w\s]', '', str(text).strip())
    text = re.sub(r'\d',' ',text) #matches any digit from 0 to 100000..., \Dmatches non-digits
    text = re.sub(r'\s+',' ',text) #\s matches any whitespace, \s+ matchesmultiple whitespace, \S matches non-whitespace

    return text

In [13]:
nltk.download('punkt')
!pip install sastrawi

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.0 MB/s eta 0:00:00


In [14]:
from spacy.lang.id import Indonesian
import spacy
nlp = Indonesian() # use directly
nlp = spacy.blank('id') # blank instance'
stopwords = nlp.Defaults.stop_words

additional_stopwords = ['scroll', 'to', 'continue','with','content']

# Menambahkan kata-kata ke dalam stopwords
for word in additional_stopwords:
    nlp.Defaults.stop_words.add(word)
    nlp.vocab[word].is_stop = True

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [15]:
def tokenisasi(text):
    tokens = text.split(" ")
    return tokens

def stemming(text, stemmer):
    # stemming process
    output = stemmer.stem(text)
    return output

def stemming_stopword_elim(text, stopwords, stemmer):
    output = ""
    for token in tokenisasi(text):
        if not token in stopwords:
            output = output + stemming(token, stemmer) + " "
    return output[:-1]

def finalpreprocess(string, stopwords, stemmer):
    if pd.notna(string):  # Memeriksa apakah nilai tidak NaN
        return stemming_stopword_elim(preprocess(string), stopwords, stemmer)
    else:
        return None  # Atau sesuaikan dengan nilai default atau tindakan lainnya sesuai kebutuhan

In [16]:
df['Cleaned Teks Berita'] = df['Berita'].apply(lambda x:finalpreprocess(x, stopwords, stemmer))
df.head()

<ipython-input-16-037a435e6051>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cleaned Teks Berita'] = df['Berita'].apply(lambda x:finalpreprocess(x, stopwords, stemmer))


,Judul,Tanggal,Link,Berita,Kode,Label,Cleaned Teks Berita
0,"ISHG Dibuka di Zona Hijau, Kembali ke Zona 7.2...",22 Mei 2024 09:13,https://finance.detik.com/bursa-dan-valas/d-73...,Indeks Harga Saham Gabungan (IHSG) pagi ini di...,1,ihsg naik,indeks harga saham gabung ihsg pagi buka zona ...
1,IHSG Dibuka di Zona Hijau Usai Kemarin Melempem,21 Mei 2024 09:12,https://finance.detik.com/bursa-dan-valas/d-73...,Indeks Harga Saham Gabungan (IHSG) pagi ini di...,1,ihsg naik,indeks harga saham gabung ihsg pagi buka zona ...
2,"Dibuka di Zona Hijau, IHSG Sore Ini Ditutup Me...",20 Mei 2024 16:31,https://finance.detik.com/bursa-dan-valas/d-73...,Indeks Harga Saham Gabungan (IHSG) menutup per...,0,ihsg turun,indeks harga saham gabung ihsg tutup dagang le...
3,"IHSG Begitu Cerah, Awali Pekan di Zona Hijau",20 Mei 2024 09:10,https://finance.detik.com/bursa-dan-valas/d-73...,Indeks Harga Saham Gabungan (IHSG) mengawali p...,1,ihsg naik,indeks harga saham gabung ihsg awal pekan zona...
4,Akhir Pekan IHSG Betah di Zona Hijau,17 Mei 2024 17:08,https://finance.detik.com/bursa-dan-valas/d-73...,Indeks harga saham gabungan (IHSG) sepanjang h...,1,ihsg naik,indeks harga saham gabung ihsg zona hijau harg...


In [18]:
df.to_excel("data_ml_filtered.xlsx")